##  Using Python to search and download Sentinel data

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import xarray
import pandas as pd

"Sentinelsat makes searching, downloading and retrieving the metadata of Sentinel satellite images from the Copernicus Open Access Hub easy." From <a target="_blank" href="https://sentinelsat.readthedocs.io/en/stable/" >Sentinelsat — Sentinelsat 1.1.1 documentation </a>

- connect to the Python API 

In [2]:
api = SentinelAPI('username', 'password', 'https://apihub.copernicus.eu/apihub')

- search by polygon, time, and SciHub query keywords

I've created a simple ROI (rectangle) as geojson file, called map.geojson in order to intersect the footprint with data to search.

In [3]:
footprint = geojson_to_wkt(read_geojson('../sentinel1/map.geojson'))
products = api.query(footprint,
                     date = ('20180605', '20180610'),
                     platformname = 'Sentinel-1',
                     producttype='SLC')

A SentinelAPI object can allow different parameters. Here it is date of ingestion , the type of platform and wihich kkind of product you need.

- convert to Pandas DataFrame

In [5]:
products_df = api.to_dataframe(products)

In [ ]:
products_df

## Testing products_df visualization as xarray

In [10]:
products_df.to_xarray()

<xarray.Dataset>
Dimensions:                      (index: 1)
Coordinates:
  * index                        (index) object 'b5c7ca6d-768e-4600-a2dd-556d...
Data variables: (12/42)
    title                        (index) object 'S2B_MSIL2A_20220430T100019_N...
    link                         (index) object "https://apihub.copernicus.eu...
    link_alternative             (index) object "https://apihub.copernicus.eu...
    link_icon                    (index) object "https://apihub.copernicus.eu...
    summary                      (index) object 'Date: 2022-04-30T10:00:19.02...
    ondemand                     (index) object 'false'
    ...                           ...
    platformserialidentifier     (index) object 'Sentinel-2B'
    processinglevel              (index) object 'Level-2A'
    datastripidentifier          (index) object 'S2B_OPER_MSI_L2A_DS_2BPS_202...
    granuleidentifier            (index) object 'S2B_OPER_MSI_L2A_TL_2BPS_202...
    identifier                   (index) object 'S2B_MSIL2A_20220430T100019_N...
    uuid                         (index) object 'b5c7ca6d-768e-4600-a2dd-556d...

## Testing Download API by using the id product

Before download sentinel data, control if the product is Online and able to directly download. Otherwise you must unblock the product by swutching it from OFFLINE status to ONLINE status.

In [14]:
product_info = api.get_product_odata('8a464f41-cb80-4350-848f-a95da9810991')
product_info['Online']

False

In [39]:
api.download('8a464f41-cb80-4350-848f-a95da9810991')

MD5 checksumming:   0%|          | 0.00/4.50G [00:00<?, ?B/s]

{'id': '8a464f41-cb80-4350-848f-a95da9810991',
 'title': 'S1B_IW_SLC__1SDV_20180607T041420_20180607T041448_011262_014ABB_AC0E',
 'size': 4496221397,
 'md5': 'cd13a4fe35c5c197ca83c75c7acf036f',
 'date': datetime.datetime(2018, 6, 7, 4, 14, 20, 208000),
 'footprint': 'POLYGON((28.414934 37.326553,25.574972 37.724804,25.919802 39.401939,28.828909 39.004982,28.414934 37.326553))',
 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8a464f41-cb80-4350-848f-a95da9810991')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2018, 6, 7, 9, 21, 30, 469000),
 'Ingestion Date': datetime.datetime(2018, 6, 7, 9, 13, 12, 2000),
 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('8a464f41-cb80-4350-848f-a95da9810991')/Products('Quicklook')/$value",
 'path': 'S1B_IW_SLC__1SDV_20180607T041420_20180607T041448_011262_014ABB_AC0E.zip',
 'downloaded_bytes': 4496221397}

In this case I select a product offline. It exists an API (trigger_offline_retrieval) in which we pass the id product in order to re-activate the product in 24h. The response of server is a boolean (True) for the success of request. Then you could try to download your desired data.

In [6]:
api.trigger_offline_retrieval('8a464f41-cb80-4350-848f-a95da9810991')

True

## Displaying product info 

In [16]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 87438c76-c585-43d6-875d-490056fab559 to be152722-3387-4d7c-a9fa-0ce74f824e9c
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    4 non-null      object        
 1   link                     4 non-null      object        
 2   link_alternative         4 non-null      object        
 3   link_icon                4 non-null      object        
 4   summary                  4 non-null      object        
 5   ondemand                 4 non-null      object        
 6   ingestiondate            4 non-null      datetime64[ns]
 7   beginposition            4 non-null      datetime64[ns]
 8   endposition              4 non-null      datetime64[ns]
 9   missiondatatakeid        4 non-null      int64         
 10  orbitnumber              4 non-null      int64         
 11  lastorbitnumber          4 non-null 

If we want to display all the information within the columns of Panda Dataframe, we must set the max number of columns we need to visualize. I set an hight number in order to all visualize.

In [8]:
pd.set_option('display.max_columns', 100)

In [ ]:
products_sorted

Sentinelsat allows filtering and sorting of search results before download.

In [ ]:
products_sorted = products_df.sort_values(['size'], ascending=[True])
products_sorted